7장 딥러닝을 위한 고급 도구
------

### 7.1 Sequential 모델을 넘어서 : 케라스의 함수형 API

지금까지 모든 신경망은 Sequential 모델을 사용하여 만듬

일부 네트워크는 개별 입력이 여러 개 필요하거나 출력이 여러 개 필요. 층을 차례대로 쌓지 않고 층 사이를 연결하여 그래프처럼 만드는 네트워크도 있다.

다중 입력 모델 예시
- 모듈 병합
  - 완전 연결 모듈 - 메타데이터
  - RNN 모듈 - 텍스트 설명
  - 컨브넷 모듈 - 사진
- 가중치를 구해서 가격 예측

최근에 개발된 많은 신경망 구조는 선형적이지 않은 네트워크 토폴로지(topology)가 필요
- 빈순환 유향 그래프 같은 네트워크 구조

![alt text](https://www.researchgate.net/profile/Bo_Zhao48/publication/312515254/figure/fig3/AS:489373281067012@1493687090916/nception-module-of-GoogLeNet-This-figure-is-from-the-original-paper-10.png)

최근에는 모델에 잔차 연결을 추가하는 경향도 있음

![alt text](https://miro.medium.com/max/6652/1*OFfO8VzLv8GNFNRKafvB7w.png)

이런 경우는 케라스의 Sequential 클래스를 사용해서는 만들지 못 함

함수형 API를 사용하여 만들 수 있음

#### 7.1.1 함수형 API 소개

함수형 API에서는 직접 텐서들의 입출력을 다룸
- 함수처럼 층을 사용하여 텐서를 입력받고 출력함 -> 그래서 함수형 API라고 부름

In [3]:
from keras import Input, layers

input_tensor = Input(shape=(32, )) # 텐서
dense = layers.Dense(32, activation='relu') # 함수처럼 사용하기 위해 층 객체를 만듬

output_tensor = dense(input_tensor) # 텐서와 함께 층을 호출하면 텐서를 반환

Sequential 모델과 함수형 API로 만든 동일한 모델을 나란히 비교

In [4]:
from keras.models import Sequential, Model
from keras import layers
from keras import Input

seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64, )))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

# 함수형 API를 만든 모델
input_tensor = Input(shape=(64, ))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

# 입력과 출력 텐서를 지정하여 Model 클래스의 객체를 만듬
model = Model(input_tensor, output_tensor)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_6 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_7 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [5]:
model.compile(
    optimizer='rmsprop', loss='categorical_crossentropy'
)

import numpy as np

x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))

model.fit(x_train, y_train, epochs=10, batch_size=128)

Epoch 1/10
1000/1000 [==============================] - 0s 239us/step - loss: 12.2830
Epoch 2/10
1000/1000 [==============================] - 0s 14us/step - loss: 14.2161
Epoch 3/10
1000/1000 [==============================] - 0s 10us/step - loss: 17.6463
Epoch 4/10
1000/1000 [==============================] - 0s 12us/step - loss: 22.1052
Epoch 5/10
1000/1000 [==============================] - 0s 11us/step - loss: 27.7020
Epoch 6/10
1000/1000 [==============================] - 0s 11us/step - loss: 34.2840
Epoch 7/10
1000/1000 [==============================] - 0s 11us/step - loss: 41.4813
Epoch 8/10
1000/1000 [==============================] - 0s 14us/step - loss: 49.6167
Epoch 9/10
1000/1000 [==============================] - 0s 13us/step - loss: 58.7254
Epoch 10/10
1000/1000 [==============================] - 0s 9us/step - loss: 68.8574


In [6]:
score=model.evaluate(x_train, y_train)

1000/1000 [==============================] - 0s 35us/step


In [7]:
score

74.94670776367188

#### 7.1.2 다중 입력 모델

함수형 API는 다중 입력 모델을 만든느 데 사용할 수 있다ㅏ.

일반적으로 이런 모델은 서로 다른 입력 가지를 합치기 위해 여러 텐서를 연결할 수 있는 층을 사용
- 텐서를 더하거나 이어 붙이는 식

```python
keras.layers.add, keras.layers.concatenate
```

question-answering 모델을 이용해서 확인해보자
- 질문-응답 모델은 2개의 입력을 가짐
- 하나는 자연어 질문, 또 하나는 답변에 필요한 정보가 담겨 있는 텍스트(예를 들어 뉴스 기사)
- 가장 간단한 구조는 미리 정의한 어휘 사전에서 소프트맥스 함수를 통해 한 단어로 된 답을 출력


응답
- Dense
- concatenate
  - LSTM - Embedding - 참고 텍스트
  - LSTM - Embedding - 질문

In [12]:
from keras.models import Model
from keras import layers
from keras import Input

text_voca_size = 10000
question_voca_size = 10000
answer_voca_size = 500

# 텍스트 입력은 길이가 정해지지 않은 정수 시퀀스. 입력 이름을 지정할 수 있다.
text_input = Input(shape=(None, ), dtype='int32', name='text')

# 입력을 크기가 64인 벡터의 시퀀스로 임베딩
embedded_text = layers.Embedding(text_voca_size, 64)(text_input)

# LSTM을 사용하여 이 벡터들을 하나의 벡터로 인코딩
encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None, ), dtype='int32', name='question')
embedded_question = layers.Embedding(question_voca_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

# 인코딩된 질문과 텍스트를 연결
concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

answer = layers.Dense(answer_voca_size, activation='softmax')(concatenated)

# 모델 객체를 만들고 2개의 입력과 출력을 주입
model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [13]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               (None, None)         0                                            
__________________________________________________________________________________________________
question (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, None, 64)     640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, None, 32)     320000      question[0][0]                   
____________________________________________________________________________________________

이 모델은 입력이 2개인데 어떻게 훈련을 할까? 두 가지 방법이 존재
- 넘파이 배열의 리스트를 주입
- 입력 이름과 넘파이 배열로 이루어진 딕셔너리를 모델의 입력으로 주입

In [15]:
import numpy as np
from keras.utils import to_categorical

num_samples = 1000
max_length = 100

# 랜덤한 넘파이 데이터를 생성
text = np.random.randint(1, text_voca_size, size=(num_samples, max_length))
question = np.random.randint(1, question_voca_size, size=(num_samples, max_length))

answers = np.random.randint(0, answer_voca_size, size=num_samples)
# 답은 정수가 아닌 원-핫 인코딩된 벡터
answers = to_categorical(answers)

# 리스트 입력을 사용하여 학습
model.fit([text, question], answers, epochs=10, batch_size=128)

# model.fit({'text' : text, 'question' : question}, answers, epochs=10, batch_size=128)

/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
1000/1000 [==============================] - 1s 1ms/step - loss: 6.2145 - acc: 0.0010
Epoch 2/10
1000/1000 [==============================] - 1s 518us/step - loss: 6.1977 - acc: 0.0490
Epoch 3/10
1000/1000 [==============================] - 0s 466us/step - loss: 6.1580 - acc: 0.0270
Epoch 4/10
1000/1000 [==============================] - 0s 463us/step - loss: 6.0533 - acc: 0.0030
Epoch 5/10
1000/1000 [==============================] - 0s 460us/step - loss: 5.9919 - acc: 0.0050
Epoch 6/10
1000/1000 [==============================] - 0s 469us/step - loss: 5.8978 - acc: 0.0080
Epoch 7/10
1000/1000 [==============================] - 0s 463us/step - loss: 5.7944 - acc: 0.0160
Epoch 8/10
1000/1000 [==============================] - 0s 474us/step - loss: 5.7109 - acc: 0.0320
Epoch 9/10
1000/1000 [==============================] - 0s 489us/step - loss: 5.6324 - acc: 0.0430
Epoch 10/10
1000/1000 [==============================] - 0s 485us/step - loss: 5.5466 - acc: 0.0500


#### 7.1.3 다중 출력 모델

다중 출력 모델을 만들어 보자
- 소셜 미디어에서 익명 사용자의 포스트를 입력으로 받아 그 사람의 나이, 성별, 소득 수준 등을 예측

In [16]:
from keras.models import Model
from keras import layers
from keras import Input

voca_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None, ), dtype='int32', name='posts')
embedded_posts = layers.Embedding(voca_size, 256)(posts_input)

x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)

x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)

x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPool1D()(x)
x = layers.Dense(128, activation='relu')(x)

# 출력 중에 이름을 지정
age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups, activation='softmax', name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])

model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
posts (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, None, 256)    12800000    posts[0][0]                      
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, None, 128)    163968      embedding_8[0][0]                
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, None, 128)    0           conv1d_1[0][0]                   
____________________________________________________________________________________________

모델을 훈련하려면 네트워크 출력마다 다른 손실 함수를 지정해야함
- 손실 값을 합치는 가장 간단한 방법은 모두 더하는 것
- compile 메소드를 이용해서 처리

In [17]:
model.compile(optimizer='rmsprop', loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'])
# model.compile(optimizer='rmsprop', loss={'age' : 'mse', 'income' : 'categorical_crossentropy', 'gender' : 'binary_crossentropy'})

손실 값이 많이 불균형하면 모델이 개별 손실이 가장 큰 작업에 치우쳐 표현을 최적화 함
- 다른 작업들은 손해를 입음
- 이를 해결하기 위해 손실 값이 최종 손실에 기여하는 수준을 지정할 수 있음
  - 손실 값의 스케일이 다를 때 유용
- mse는 일반적으로 3~5사이의 값을 가짐, 성별 분류는 0.1
  - 각각 0.25, 10의 가중치를 줌

In [18]:
model.compile(
    optimizer='rmsprop', 
    loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'],
    loss_weight=[0.25, 1., 10.]
)

# model.compile(
#     optimizer='rmsprop', 
#     loss={'age' : 'mse', 'income' : 'categorical_crossentropy', 'gender' : 'binary_crossentropy'},
#     loss_weight={
#         'age':0.25,
#         'income' : 1.,
#         'gender' : 10.
#     }
# )

In [ ]:
# 데이터 있다고 가정하고 짠 코드

# model.fit(posts, [age_targets, income_targets, gender_targets], epochs=10, batch_size=64)

#### 7.1.4 층으로 구성된 비순환 유향 그래프(DAG)

함수형 API를 사용하면 다중 입력이나 다중 출력 모델뿐만 아니라 내부 토폴로지가 복잡한 네트워크도 만들 수 있다.
- 비순환 유향 그래프(DAG)
- 비순환이라는 것이 중요
  - 다시 말해 이 그래프는 원형을 띨 수 없음 -> 텐서 x가 자기 자신을 출력하는 층의 입력이 될 수 없다.
  - 만들 수 있는 루프(순환 연결)는 순환 층의 내부에 있는 것뿐

가장 유명한 2개는 인셉션 모듈과 잔차 연결

#### 인셉션 모듈

Inception은 합성곱 신경망에서 인기 있는 네트워크 구조
- 네트워크 안의 네트워크라는 구조에 영감을 받은 구조
- 가장 기본적인 인셉션 모듈 형태는 3~4개의 가지를 가짐
- 네트워크가 따로따로 공간 특성과 채널 방향의 특성을 학습하도록 도움
- 한꺼번에 학습하는 것보다 효과가 더 높음
  - 더 복잡한 인셉션은 풀링 연산, 여러가지 합성곱 사이즈, 공간 합성곱이 없는 가지 등등

![alt text](https://images.deepai.org/django-summernote/2019-06-18/2cec735b-2347-4ded-ae2b-e8a8384f7b46.png)

In [ ]:
# 4D 텐서가 입력된다고 가정

from keras import layers

# 모든 가지는 동일한 스트라이드(2)를 사용. 출력 크기를 동일하게 만들어 하나로 합치기 위함
branch_a = layers.Conv2D(128, 1, activation='relu', strides=2)(x)

branch_b = layers.Conv2D(128, 1, activation='relu')(x)
branch_b = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_b)

branch_c = layers.AveragePooling2D(3, strides=2)(x)
branch_c = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_c)

branch_d = layers.Conv2D(128, 1, activation='relu')(x)
branch_d = layers.Conv2D(128, 3, activation='relu')(branch_d)
branch_d = layers.Conv2D(128, 3, activation='relu')(branch_d)

# axis=-1은 현재 배열의 마지막 axis를 의미합니다.
# http://taewan.kim/post/numpy_cheat_sheet/

output = layers.concatenate([branch_a, branch_b, branch_c, branch_d], axis=-1)

In [21]:
import keras

In [22]:
model = keras.applications.inception_v3.InceptionV3()

96116736/96112376 [==============================] - 20s 0us/step


In [24]:
# model.summary()

#### 잔차 연결

residual connection은 Xception을 포함하여 2015년 이후 등장한 많은 네트워크 구조에 있는 그래프 형태의 네트워크 컴포넌트
- 그래디언트 소실과 representational bottleneck을 해결
- 10개 층 이상을 가진 모델에 잔차 연결을 추가하면 도움이 됨

잔차 연결은 하위 층의 출력을 상위 층의 입력으로 사용
- 순서대로 놓인 네트워크를 질러가는 연결이 만들어짐
- 하위 층의 출력이 상위 층의 활성화 출력에 연결되는 것이 아니고 더해짐
- 따라서 두 출력의 크기가 동일해야 함
- 크기가 다르면 선형 변환을 사용하여 하위층의 활성화 출력을 목표 크기로 변환
  - 활성화 함수를 사용하지 않는 Dense층이나 합성곱의 특성 맵이라면 활성화 함수가 없는 1 X 1 합성곱

In [ ]:
# 케라스에서 특성 맵의 크기가 같을 때 원본을 그대로 사용하는 잔차 연결을 구현한 예
# 입력 x는 4D 텐서라고 가정

from kerase import layers

x = ...
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x) # x에 변환을 적용
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)

y = layers.add([y, x]) # 원본 x를 출력 특성에 더함

In [ ]:
# 다음은 특성 맵의 크기가 다를 떄 선형 변환을 사용하여 잔차 연결을 구현한 예

from kerase import layers

x = ...
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x) # x에 변환을 적용
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.MaxPooling2D(2, strides=2)(y)

# y와 크기를 맞추기 위해 1x1 합성곱을 사용하여 원본 텐서 x를 다운샘플링 함
residual = layers.Conv2D(128, 1, strides=2, padding='same')(x)

y=layers.add([y, residual]) # 다운샘플링된 x를 출력 특성에 더하기

#### 7.1.5 층 가중치 공유

함수형 API의 중요한 또 하나의 기능은 층 객체를 여러 번 재사용할 수 있다는 것
- 층 객체를 두 번 호출하면 새로운 층 객체를 만들지 않고 각 호출에 동일한 가충치를 재사용
- 공유 가지를 가진 모델을 만들 수 있음

두 문장 사이의 의미가 비슷한지 측정하느 모델 예시
- 두 개의 입력(비교할 두 개의 문장)
- 0(관련 없는 문장)과 1(동일하거나 재구성) 사이의 점수 출력
- 자연어 질의에 대한 중복 제거를 포함하여 많은 애플리케이션에서 유용하게 사용

A에서 B에 대한 유사도는 B에서 A에 대한 유사도와 같다
- 입력 시퀀스가 바뀌어도 됨
- 각 입력 문장을 2개의 독립된 모델에서 처리하는 것보다, 하나의 LSTM 층으로 양쪽을 모두 처리하는 것이 좋음
- 이 LSTM 층의 표현(가중치)은 두 입력에 대해 함께 학습
- 이를 Siamese LSTM 모델(샴 LSTM) 또는 공유 LSTM 이라고 부름

In [ ]:
from keras.models import Model
from keras import layers
from keras import Input

# LSTM 객체 생성
lstm = layers.LSTM(32)

# 모델의 왼쪽 가지 구성, 입력은 크기가 128인 벡터의 가변 길이 시퀀스
left_input = Input(shape=(None, 128))
left_output = lstm(left_input)

# 모델의 오른쪽 가지 구성, 기존 층 객체를 호출하면 가중치가 재사용됨
right_input = Input(shape=(None, 128))
right_output = lstm(right_input)

# 맨 위에 분류기
merged = layers.concatenate([left_output, right_output], axis=-1)
predictions = layers.Dense(1, activation='sigmoid')(merged)

# 모델 객체를 만들고 훈련, 이런 모델을 훈련하면 LSTM 층의 가중치는 양쪽 입력을 바탕으로 업데이트
model = Model([left_output, right_output], predictions)
model.fit([left_output, right_output], targets)

#### 7.1.6 층과 모델

함수형 API 에서는 모델을 층처럼 사용할 수 있음
- 모델을 '커다란 층'으로 생각해도 됨
- 입력 텐서로 모델을 호출해서 출력 텐서를 얻을 수 있다는 뜻

```python
y = model(x)

# 입력 텐서, 출력 텐서가 여러 개이면 텐서의 리스트로 호출

y1, y2 = model([x1, x2])
```

모델 객체를 호출할 때 모델의 가중치가 재사용됨
- 층 객체 호출 할 때와 동일한 개념

In [ ]:
from keras.models import Model
from keras import layers
from keras import Input

# 이미지 처리 기본 모델 = Xception Network(합성곱 기반 층만 사용)
xception_base = application.Xception(weights=None, include_top=False)

# 입력은 250 x 250 RGB
left_input = Input(shape=(250, 250, 3))
right_input = Input(shape=(250, 250, 3))

# 같은 비전 모델을 두 번 호출
left_features = xception_base(left_input)
right_features = xception_base(right_input)

merged_features = layers.concatenate([left_features, right_features], axis=-1)

#### 7.1.7. 정리
- 차례대로 층을 쌓는 것 이상이 필요할 때는 Sequential API를 사용하지 않음
- 함수형 API를 사용하여 다중 입력, 다중 출력, 복잡한 네트워크 토폴로지를 갖는 케라스 모델을 만드는 방법
- 다른 네트워크 기지에서 같은 층이나 모델 객체를 여러 번 호출하여 가중치를 재사용하는 방법

### 7.2 케라스 콜백과 텐서보드를 사용한 딥러닝 모델 검사와 모니터링

#### 7.2.1 콜백을 사용하여 모델의 훈련 과정 제어하기

모델을 훈련할 때 미리 예상할 수 없는 것들이 많음
- 최적의 에포크는 얼마인가?
- 기타 등등....낭비가 많음

더 좋은 방법은 검증 손실이 더 이상 향상되지 않을 떄 훈련을 멈추는 것
- 케라스 콜백을 사용하여 구현 가능
- 콜백은 모델의 fit() 메서드가 호출될 때 전달되는 객체(특정 매서드를 구현한 클래스 객체)
- 훈련하는 동안 모델은 여러 지점에서 콜백을 호출
- 콜백은 모델의 상태와 성능에 대한 모든 정보에 접근하고 훈련 중지, 모델 저장, 가중치 적재 또는 모델 상태 변경 등을 처리할 수 있음

콜백을 사용하는 예시
- 모델 체크포인트 저장 : 훈련하는 동안 어떤 지점에서 모델의 현재 가중치를 저장
- 조기 종료(early stoping) : 검증 손실이 더 이상 향상되지 않을 때 훈련을 중지(물론 훈련하는 동안 얻은 가장 좋은 모델을 저장)
- 훈련하는 동안 하이퍼파라미터 값을 동적으로 조정 : 옵티마이저의 학습률
- 훈련과 검증 지표를 로그에 기록하거나 모델이 학습한 표현이 업데이트될 때마다 시각화 : 케라스의 진행 표시줄(progress bar)이 하나의 콜백

**ModelCheckpoint와 EarlyStopping 콜백**

EarlyStopping 콜백
- 정해진 에포크 동안 모니터링 지표가 향상되지 않을 때 훈련을 중지
- 일반적으로 이 콜백은 훈련하는 동안 모델을 계속 저장해 주는 ModelCheckpoint와 함께 사용

In [27]:
import keras

# fit() 메서드의 callbacks 매개변수를 사용하여 콜백의 리스트를 모델로 전달. 몇 개의 콜백이라도 전달할 수 있음
callbacks_list = [
    keras.callbacks.EarlyStopping( # 성능 향상 멈추면 훈련 중지
        monitor='val_acc', # 모델의 검증 정확도를 모니터링
        patience=1, # 1 에포크보다 더 길게(즉 2 에포크 동안) 정확도가 향상되지 않으면 훈련이 중지
    ),
    keras.callbacks.ModelCheckpoint( # epoch 마다 현재 가중치를 저장
        filepath='my_model.h5', # 모델 파일의 경로
        monitor='val_loss', # val_loss가 좋아지지 않으면 모델 파일을 덮어쓰지 않는다는 뜻. 훈련하는 동안 가장 좋은 모델이 저장
        save_best_only=True,
    )
]


model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['acc']
)


model.fit(
    x,y,
    epochs=10,
    batch_size=32,
    callbacks=callbacks_list,
    validation_data=(x_val, y_val)
)

ReduceLROnPlateau 콜백
- 검증 손실이 향상되지 않을 때 학습률을 작게 할 수 있음
- 손실 곡선이 평탄할 때 학습률을 작게 하거나 크게 하면 훈련 도중 지역(local) 최솟값에서 효과적으로 빠져나올 수 있음

In [ ]:
callbacks_list = [
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', # 모델의 검증 손실을 모니터링
        factor=0.1, # 콜백이 호출될 때 학습률을 10배로 줄임
        patience=10, # 검증 손실이 10 에포크 동안 좋아지지 않으면 콜백이 호출됨
    )
    
]

model.fit(
    x,y,
    epochs=10,
    batch_size=32,
    callbacks=callbacks_list,
    validation_data=(x_val, y_val)
)

자신만의 콜백 만들기도 가능하다 - 책 참조

#### 7.2.2 텐서보드 소개 : 텐서플로우 시각화 프레임워크

설명 작성 중

In [70]:
import keras
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import callbacks

In [71]:
max_features = 2000
max_len = 100

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

model = Sequential()
model.add(layers.Embedding(max_features, 128, input_length=max_len, name='embed'))

model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))

model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(1))

model.summary()
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 100, 128)          256000    
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 94, 32)            28704     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 18, 32)            0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 12, 32)            7200      
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 291,937
Trainable params: 291,937
Non-trainable params: 0
________________________________________________

In [72]:
current_directory_path = os.getcwd()
log_directory_path = current_directory_path + os.sep + 'my_log_dir'

In [74]:
callbacks_value = [
    callbacks.TensorBoard(
        log_dir=log_directory_path, # 로그 파일이 기록될 위치
        histogram_freq=1, # 1 에포크마다 활성화 출력의 히스토그램을 기록
        embeddings_freq=1, # 1 에포크마다 임베딩 데이터를 기록
    )
    
]

history=model.fit(
    x_train, y_train,
    epochs=20,
    batch_size=128,
    validation_split=0.2,
    callbacks=callbacks_value
)

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 7s 333us/sample - loss: 2.0056 - acc: 0.6403 - val_loss: 0.4883 - val_acc: 0.7772
Epoch 2/20
20000/20000 [==============================] - 6s 301us/sample - loss: 0.4808 - acc: 0.8123 - val_loss: 0.5469 - val_acc: 0.7868
Epoch 3/20
20000/20000 [==============================] - 6s 319us/sample - loss: 0.4305 - acc: 0.8437 - val_loss: 0.4818 - val_acc: 0.8082
Epoch 4/20
20000/20000 [==============================] - 6s 303us/sample - loss: 0.3609 - acc: 0.8741 - val_loss: 0.8200 - val_acc: 0.7832
Epoch 5/20
20000/20000 [==============================] - 7s 333us/sample - loss: 0.3217 - acc: 0.8949 - val_loss: 0.6512 - val_acc: 0.8110
Epoch 6/20
20000/20000 [==============================] - 6s 299us/sample - loss: 0.2785 - acc: 0.9232 - val_loss: 0.7543 - val_acc: 0.8124
Epoch 7/20
20000/20000 [==============================] - 6s 295us/sample - loss: 0.2244 - acc: 0.9444 - val_lo

### 7.3 모델의 성능을 최대로 끌어올리기

#### 7.3.1 고급 구조 패턴

**배치 정규화**

정규화는 머신 러닝 모델에 주입되는 샘플들을 균일하게 만드는 광범위한 방법
- (데이터 - 평균) / 표준편차, 분산 1


```python
normalized_data = (data - np.mean(data, axis=...)) / np.std(data, axis=...)
```

이전에는 모델에 데이터를 주입하기 전에 정규화 했음
- 하지만 데이터 정규화는 네트워크에서 일어나는 모든 변환 후에도 고려되어야 함
- 쉽게 말해 입력 할 때 들어가는 데이터가 정규화 되어 있더라도, 출력 되는 데이터가 정규화 되어 있다는 보장 못 함


Batch normalization
- 훈련하는 동안 평균과 분산이 바뀌더라도 이에 적응하여 데이터를 정규화 함
- 훈련 과정에 사용된 배치 데이터의 평균과 분산에 대한 지수 이동 평균을 내부에 유지
- 배치 정규화의 주요 효과는 잔차 연결과 매우 흡사하게 gradient의 전파를 도와주는 것
- 결국 더 깊은 네트워크를 구성할 수 있음
- 매우 깊은 네트워크라면 여러 개의 BatchNormalization 층을 포함해야 훈련할 수 있음
- BatchNormalization 층은 일반적으로 합성곱이나 완전 연결 층 다음에 사용

```python
# Conv2D 층 다음에
conv_model.add(layers.Conv2D(32, 3, activation='relu'))
conv_model.add(layers.BatchNormalization())

# Dense 층 다음에
dense_model.add(layers.Dense(32, activation='relu'))
dense_model.add(layers.BatchNormalization())
```

BatchNormalization 클래스에는 정규화할 특서 축을 지정하는 axis 매개변수가 있음
- 매개변수 기본값은 입력 텐서의 마지막 축을 나타내는 -1
> axis: Integer, the axis that should be normalized
        (typically the features axis).
        For instance, after a `Conv2D` layer with
        `data_format="channels_first"`,
        set `axis=1` in `BatchNormalization`.

In [75]:
keras.layers.BatchNormalization?

Init signature:
keras.layers.BatchNormalization(
    axis=-1,
    momentum=0.99,
    epsilon=0.001,
    center=True,
    scale=True,
    beta_initializer='zeros',
    gamma_initializer='ones',
    moving_mean_initializer='zeros',
    moving_variance_initializer='ones',
    beta_regularizer=None,
    gamma_regularizer=None,
    beta_constraint=None,
    gamma_constraint=None,
    **kwargs,
)
Docstring:     
Batch normalization layer (Ioffe and Szegedy, 2014).

Normalize the activations of the previous layer at each batch,
i.e. applies a transformation that maintains the mean activation
close to 0 and the activation standard deviation close to 1.

# Arguments
    axis: Integer, the axis that should be normalized
        (typically the features axis).
        For instance, after a `Conv2D` layer with
        `data_format="channels_first"`,
        set `axis=1` in `BatchNormalization`.
    momentum: Momentum for the moving mean and the moving variance.
    epsilon: Small float added to var

**깊이별 분리 합성곱**

Conv2D를 대체하면서 더 가볍고 더 빨라 모델의 성능을 높일 수 있는 층이 있...?
- 깊이별 분리 합성곱(depthwise separable convolution) - SeparableConv2D
- 이 층은 입력 채널별로 따로따로 공간 방향의 합성곱을 수행 -> 그 후 점별 합성곱(1 x 1 합성곱)을 통해 출력 채널을 합침
- 이는 공간 특성의 학습과 채널 방향 특성의 학습을 분리하는 효과를 냄
- 입력에서 공간상 위치는 상관관계가 크지만 채널별로는 매우 독립적이라고 가정한다면 타당
- 이 방법은 모델 파라미터와 연산의 수를 크게 줄여 주기 때문에 작고 더 빠른 모델을 만듬

이 장점은 제한된 데이터로 작은 모델을 처음부터 훈련시킬 때 특히 더 중요
- 작은 데이터셋에서 이미지 분류 문제(소프트맥스 분류)를 위한 가벼운 깊이별 분리 컨브넷 예시...아래 보자

```python
height, width, channels, num_classes = 64, 64, 3, 10

model = Sequential()
model.add(layers.SeparableConv2D(
    32, 3, activation='relu', input_shape=(height, width, channels,)
))
model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.MaxPooling2D(2))

model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.SeparableConv2D(128, 3, activation='relu'))
model.add(layers.MaxPooling2D(2))

model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.SeparableConv2D(128, 3, activation='relu'))
model.add(layers.GlobalAveragePooling2D())

model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
```

#### 7.3.2 하이퍼파라미터 최적화

하루종일 하이퍼파라미터를 수정하는 것은 사람이 할 짓이 아니다. -> 기계에 위임하는 것이 더 낫다

최적화 과정
1. 일련의 하이퍼파라미터를 (자동으로) 선택
2. 선택된 하이퍼파라미터로 모델을 만듬
3. 훈련 데이터에 학습하고 검증 데이터에서 최종 성능을 측정
4. 다음으로 시도할 하이퍼파라미터를 (자동으로) 선택
5. 이 과정을 반복
6. 마지막으로 테스트 데이터에서 성능을 측정

여러 가지 기법을 사용할 수 있음
- 베이지안 최적화(bayesian optimization)
- 유전 알고리즘(genetic algorithms)
- 간단한 랜덤 탐색(random search)
- 라이브러리 : https://github.com/hyperopt/hyperopt